In [ ]:
! pip install -r requirements.txt

In [7]:
import requests

In [1]:
from bs4 import BeautifulSoup
import csv

In [10]:
event_list = []

for i in range(1, 41):
    url = f"https://visitseattle.org/events/page/{i}"
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        selector = "div.search-result-preview > div > h3 > a"
        a_eles = soup.select (selector)
        event_list += [x['href']for x in a_eles]

print('Event List:', event_list)

Event List: ['https://visitseattle.org/events/amber-liu/', 'https://visitseattle.org/events/disability-justice/', 'https://visitseattle.org/events/hughes-bros-presents/', 'https://visitseattle.org/events/sarya-wu/', 'https://visitseattle.org/events/the-sweet-lillies/', 'https://visitseattle.org/events/dinosaur-jr/', 'https://visitseattle.org/events/black-dogs/', 'https://visitseattle.org/events/blue-elephant-and-the-seven-snakes/', 'https://visitseattle.org/events/brock-lanzetti-ogawa/', 'https://visitseattle.org/events/fact-and-fiction-the-lord-of-the-rings/', 'https://visitseattle.org/events/groundation/', 'https://visitseattle.org/events/kayla-min-andrews/', 'https://visitseattle.org/events/ol-doris/', 'https://visitseattle.org/events/rosetan/', 'https://visitseattle.org/events/untold-stories-wire-life-and-art-with-kristin-tollefson/', 'https://visitseattle.org/events/y-la-bamba/', 'https://visitseattle.org/events/dead-bars/', 'https://visitseattle.org/events/emotional-a-night-for-t